Skeleton Code

In [27]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time

# Function to download and load dataset
def load_data():
    page_url = 'https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones'
    page_response = requests.get(page_url)
    if page_response.status_code == 200:
        soup = BeautifulSoup(page_response.content, 'html.parser')
        download_link = soup.select_one('a[href$=".zip"]')['href']
        full_download_url = 'https://archive.ics.uci.edu' + download_link
        response = requests.get(full_download_url)
        if response.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(response.content)) as outer_zip:
                inner_zip_name = 'UCI HAR Dataset.zip'
                with outer_zip.open(inner_zip_name) as inner_zip_file:
                    with zipfile.ZipFile(io.BytesIO(inner_zip_file.read())) as inner_zip:
                        with inner_zip.open('UCI HAR Dataset/train/X_train.txt') as myfile:
                            df = pd.read_csv(myfile, delim_whitespace=True, header=None)
                        with inner_zip.open('UCI HAR Dataset/train/y_train.txt') as myfile_y:
                            y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)
    else:
        raise Exception("Failed to download or parse the dataset.")
    return df, y

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np
import time

# Load dataset
df, y = load_data()

#TASK 1 - DO EDA and understand a little about the data.
#Only important thing is to know that it has a lot of features that don't make sense, just a
#bunch of readings from sensors.
#We think many of these features are redundant or irrelevant, and we want to find good features.

<ipython-input-27-836ccc4d8ebc>:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(myfile, delim_whitespace=True, header=None)
<ipython-input-27-836ccc4d8ebc>:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 561 entries, 0 to 560
dtypes: float64(561)
memory usage: 31.5 MB


In [30]:
y.value_counts()

,count
0,
6,1407
5,1374
4,1286
1,1226
2,1073
3,986


In [31]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892


In [32]:
from sklearn.preprocessing import StandardScaler
st=StandardScaler()
df=st.fit_transform(df)
df

array([[ 0.20064157, -0.0636826 , -0.41962845, ..., -0.68721921,
         0.40794614, -0.00756789],
       [ 0.05594788,  0.03148567, -0.25390836, ..., -0.694138  ,
         0.40911698,  0.00787517],
       [ 0.07351535, -0.04341648, -0.07629468, ..., -0.702239  ,
         0.4102883 ,  0.02650234],
       ...,
       [-0.01566765,  0.0167814 ,  1.13222107, ..., -0.56584847,
         0.64059683,  0.34870928],
       [ 0.21586648, -0.02812252, -0.86770988, ..., -0.57766781,
         0.63147758,  0.29327564],
       [ 1.09620157,  0.12919873, -1.67268082, ..., -0.57392691,
         0.63274259,  0.33396081]])

In [33]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(df,y,test_size=0.25,random_state=1)
xtrain

array([[ 2.25982767,  0.13646002,  0.11571968, ..., -0.19792178,
         0.92243948,  0.83135764],
       [ 0.04411624, -0.2277251 ,  0.19175243, ..., -0.8711634 ,
         0.05333659,  0.41980263],
       [ 0.04192655,  0.02572733, -0.02100604, ..., -0.68293517,
         0.44876846,  0.04359406],
       ...,
       [ 0.02640686, -0.02025411,  0.04242439, ..., -0.32376084,
         0.80382605,  0.73905251],
       [ 0.03942031,  0.08444794, -0.0541105 , ..., -0.47755521,
        -0.1868364 , -0.32803986],
       [-0.21800208,  0.0244102 ,  0.221871  , ...,  1.83759575,
        -2.57910916, -0.7687313 ]])

In [34]:
ytrain

,0
4316,1
574,4
5281,5
2781,6
5455,2
...,...
905,6
5192,1
3980,5
235,4


In [35]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
ypred=model.fit(xtrain,ytrain)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [36]:
ypred=model.predict(xtest)

In [37]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest,ypred)

0.7426550598476604

In [38]:
import time
start_time = time.time()
model.fit(xtrain, ytrain)
end_time = time.time()
print(f"Training time: {end_time - start_time:.4f} seconds")
start_time = time.time()
ypred = model.predict(xtest)
end_time = time.time()
print(f"Prediction time: {end_time - start_time:.4f} seconds")

Training time: 0.0520 seconds
Prediction time: 0.0380 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [39]:
# SOLUTION -  K-Means for dimensionality reduction
n_clusters = 50
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
kmeans.fit(df_scaled.T)  # Transpose to treat features as data points
selected_features_indices = [np.random.choice(np.where(kmeans.labels_ == i)[0]) for i in range(n_clusters)]
selected_features = df_scaled[:, selected_features_indices]

In [40]:
df=selected_features
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(df,y,test_size=0.25,random_state=1)
xtrain

array([[ 0.04735575,  1.21439341,  0.29068974, ..., -0.26504537,
        -0.30885831,  0.50146345],
       [-0.23889251, -0.75093333, -0.56746463, ...,  0.56852475,
         1.30227668, -0.15461208],
       [ 0.55422333, -1.14021606, -0.57054526, ..., -0.75694642,
         1.3926496 , -0.10886135],
       ...,
       [-0.01689411, -1.23948866, -0.57096267, ..., -0.65808227,
         1.34380706,  0.88751251],
       [-0.83800588, -1.31081367, -0.57069455, ...,  1.08307775,
         0.65617997,  1.4024282 ],
       [ 0.77022388, -0.13379249, -0.56230253, ...,  1.8865672 ,
         0.29589357, -2.76117326]])

In [41]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
model.fit(xtrain,ytrain)
ypred=model.predict(xtest)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest,ypred)

0.8275299238302503